In [1]:
import pandas as pd
import numpy as np
import itertools
import urllib2
import json
from tqdm import tqdm
pd.set_option('max_columns', None)

In [2]:
week = 15

In [3]:
team_map = {1: 'Arsenal',
    2: 'Bournemouth',
    3: 'Burnley',
    4: 'Chelsea',
    5: 'Crystal Palace',
    6: 'Everton',
    7: 'Hull City',
    8: 'Leicester City',
    9: 'Liverpool',
    10: 'Manchester City',
    11: 'Manchester United',
    12: 'Middlesbrough',
    13: 'Southampton',
    14: 'Stoke City',
    15: 'Sunderland',
    16: 'Swansea City',
    17: 'Tottenham Hotspur',
    18: 'Watford',
    19: 'West Bromwich Albion',
    20: 'West Ham United'}

In [4]:
def gen_fixtures(week):
    '''Function for generating fixtures for given week'''
    
    fixurl = 'https://fantasy.premierleague.com/drf/fixtures/?event={}'
    fixlist=[]
    resp = urllib2.urlopen(fixurl.format(week))
    fixtures = json.loads(resp.read())
    resp.close()

    for fixture in fixtures:
        fixlist.append((fixture['team_h'], fixture['team_a'], fixture['kickoff_time']))

    return (pd.DataFrame(fixlist, columns=['hometeam','awayteam','time'])
     .replace(team_map))

In [5]:
def gen_ptstable(week):
    '''Function to generate points table as it stands before start of week'''
    
    taburl = 'https://footballapi.pulselive.com/football/standings?compSeasons=54&altIds=true&detail=2&FOOTBALL_COMPETITION=1&gameweekNumbers=1-{}'
        
    request = urllib2.Request(taburl.format(week-1), headers= {'Origin': 'https://www.premierleague.com'})
    contents = urllib2.urlopen(request).read()
    entry = json.loads(contents)['tables'][0]['entries']
    
    cur = []
    for row in entry:
        cur.append((row['team']['name'], int(row['overall']['points']), int(row['position'])))
    
    return pd.DataFrame(cur, columns=['team','curpts','curpos']).set_index('team')

In [6]:
# Logic contributed by Nikhil Pangarkar (@nikpan710)

def calcbestpos(team, curpos, nextfix):
    '''Function to calculate best position a team can achieve given the current 
    points table and the next round of fixtures'''
    
    teampts = curpos.loc[team, 'curpts']
    teamsahead = 0
    
    for __,fixture in nextfix.iterrows():
        
        if ((fixture['hometeam'] == team) | (fixture['awayteam'] == team)):
            if teampts + 3 < curpos.loc[opposition(team, nextfix), 'curpts']:
                teamsahead += 1
        else:
            homepts = curpos.loc[fixture.hometeam, 'curpts']
            awaypts = curpos.loc[fixture.awayteam, 'curpts']

            teamsahead_h = bestpts(homepts+3, awaypts, teampts+3)
            teamsahead_d = bestpts(homepts+1, awaypts+1, teampts+3)
            teamsahead_a = bestpts(homepts, awaypts+3, teampts+3)

            teamsahead += np.min([teamsahead_h, teamsahead_d, teamsahead_a])

    return (teamsahead + 1)

def calcworstpos(team, curpos, nextfix):
    '''Function to calculate worst position a team can end up at given the current 
    points table and the next round of fixtures'''
    
    teampts = curpos.loc[team, 'curpts']
    teamsahead = 0
    
    for __,fixture in nextfix.iterrows():
        if ((fixture['hometeam'] == team) | (fixture['awayteam'] == team)):
            if teampts <= curpos.loc[opposition(team, nextfix), 'curpts'] + 3:
                teamsahead += 1
        else:
            homepts = curpos.loc[fixture.hometeam, 'curpts']
            awaypts = curpos.loc[fixture.awayteam, 'curpts']

            teamsahead_h = worstpts(homepts+3, awaypts, teampts)
            teamsahead_d = worstpts(homepts+1, awaypts+1, teampts)
            teamsahead_a = worstpts(homepts, awaypts+3, teampts)

            teamsahead += np.max([teamsahead_h, teamsahead_d, teamsahead_a])

    return (teamsahead + 1)

def bestpts(homepts, awaypts, teampts):
    '''Function to return number of teams with better ranking than input team,
    considering the best case scenario for input team'''
    
    if ((homepts > teampts) & (awaypts > teampts)):
        return 2
    elif ((homepts <= teampts) & (awaypts <= teampts)):
        return 0
    else:
        return 1
    
def worstpts(homepts, awaypts, teampts):
    '''Function to return number of teams with better ranking than input team,
    considering the worst case scenario for input team'''
    
    if ((homepts >= teampts) & (awaypts >= teampts)):
        return 2
    elif ((homepts < teampts) & (awaypts < teampts)):
        return 0
    else:
        return 1

def opposition(team, nextfix):
    '''Function to return opposition name for input team given fixture list.'''
    
    if team in nextfix.hometeam.values:
        return nextfix.loc[nextfix.hometeam == team, 'awayteam'].values[0]
    elif team in nextfix.awayteam.values:
        return nextfix.loc[nextfix.awayteam == team, 'hometeam'].values[0]
    else:
        return np.nan

In [7]:
nextfix = gen_fixtures(week)
curpos = gen_ptstable(week)

In [8]:
posfinal = (curpos.reset_index()[['team','curpos']].
 assign(minpos = lambda x: x['team'].apply(calcbestpos, args=(curpos, nextfix)),
       maxpos = lambda x: x['team'].apply(calcworstpos, args=(curpos, nextfix)))
 )

In [9]:
posfinal.to_csv('data/posfinal_{}.csv'.format(week), index=False)
print posfinal

                    team  curpos  maxpos  minpos
0                Chelsea       1       2       1
1                Arsenal       2       4       1
2              Liverpool       3       5       2
3        Manchester City       4       5       2
4      Tottenham Hotspur       5       5       3
5      Manchester United       6      10       6
6   West Bromwich Albion       7      12       6
7                Everton       8      12       6
8             Stoke City       9      12       6
9            Bournemouth      10      12       6
10               Watford      11      12       6
11           Southampton      12      15       8
12         Middlesbrough      13      17      10
13        Crystal Palace      14      19      13
14               Burnley      15      19      13
15        Leicester City      16      19      13
16       West Ham United      17      19      13
17            Sunderland      18      20      14
18             Hull City      19      20      14
19          Swansea 

In [10]:
data = posfinal.assign(width= lambda x: x['maxpos'] - x['minpos'],
                  center= lambda x: (x['maxpos'] + x['minpos'])/2,
                  cursize= lambda x: np.where(((x['maxpos'] == x['curpos'])|(x['minpos']==x['curpos'])),
                                              20,10))

In [11]:
from bokeh.io import output_file, output_notebook, show

In [12]:
output_notebook()

Loading BokehJS ...

In [13]:
# output_file('posrange.html')

In [14]:
from bokeh.models import (ColumnDataSource, 
                          Plot, 
                          LinearAxis,
                          CategoricalAxis,
                          Range1d, 
                          FactorRange, 
                          Circle,
                          Label,
                          FixedTicker,
                          Grid,
                          Rect,
                          LabelSet,
                          Legend
                          )
# from bokeh.plotting import Figure

In [15]:
cds = ColumnDataSource(data)

p = Plot(title=None, plot_width=700, plot_height=600, min_border_top=70, 
         min_border_bottom=40, toolbar_location=None, logo=None, outline_line_color=None,
          x_range=Range1d(0,21), y_range=FactorRange(factors=list(data.team)[::-1]))

yax = CategoricalAxis(major_label_text_align='left', axis_line_color=None, 
                     major_label_text_font_size='16px', major_label_text_font_style='bold',
                     major_tick_line_color=None)
p.add_layout(yax, 'left')

p.add_glyph(cds, Rect(x='center', y='team',
                     width='width', height=0.2, 
                     line_color=None, fill_color='#9D9D9D' ))

maxp = p.add_glyph(cds, Circle(x='maxpos',y='team',fill_color='#75C5E4',line_color=None, size=20))
minp = p.add_glyph(cds, Circle(x='minpos',y='team',fill_color='#0068A1',line_color=None, size=20))
curp = p.add_glyph(cds, Circle(x='curpos',y='team',fill_color='#9D9D9D',line_color=None, size='cursize'))

labarg = dict(render_mode='canvas', text_align='center', 
                  text_baseline='middle', text_font_size='8pt',
                  text_color='#FFFFFF', text_font_style='bold', 
                  x_offset=-0.5, y_offset=-1, source=cds)

minlab = LabelSet(x='minpos', y='team', text='minpos', **labarg)
maxlab = LabelSet(x='maxpos', y='team', text='maxpos', **labarg)
p.add_layout(minlab)
p.add_layout(maxlab)

ticky = FixedTicker(ticks=[4.5, 10.5, 17.5])
tickx = FixedTicker(ticks=[16.5, 10.5, 3.5])
grx = Grid(dimension=1, ticker=tickx)
gry = Grid(dimension=0, ticker=ticky)
p.add_layout(grx)
p.add_layout(gry)

labarg2 = dict(x=5, y=10, x_units='screen', y_units='screen')
subhead1 = Label(text='Spread of possible positions following fixtures on {} - {}.'.format(
        np.min(pd.to_datetime(nextfix.time)).strftime('%b %d').replace(' 0',' '),
        np.max(pd.to_datetime(nextfix.time)).strftime('%b %d').replace(' 0',' ')),
                **labarg2)

subhead2 = Label(text='Thin grey lines indicate top four, top half and relegation zone',
                x=0, y=-10, x_units='screen', y_units='screen', text_font_size='15px')

head = Label(text_font_style='bold', text_font_size='20px', 
             text='How much could the Premier League table change?',
            **labarg2)

p.add_layout(subhead2, 'below')
p.add_layout(subhead1, 'above')
p.add_layout(head, 'above')

##Legend issues to be fixed in Bokeh 0.12.4
# leg = Legend(items=[
#         ('highest possible rise', [minp]),
#         ('lowest possible fall', [maxp]),
#         ('current position', [curp])
#     ], orientation='horizontal', location='center')
# p.add_layout(leg, 'below')

show(p)